In [58]:
import ee
import datetime
import os
import itertools
import sys
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE
import ast

In [59]:
ee.Initialize()

In [60]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('../../')

In [61]:
from BackGround_modules.Class_2_Classify_Fourier_Img   import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

##### Prepare basic parameters

In [62]:
region_cn = '中南'
region_en = 'zhongnan'

In [63]:
# define the years to be classified
year_name  = [f'{i}_{i+2}' for i in range(1990,2020,3)]

# define the number of total periods
num_invarient = 10

In [64]:
year_name

['1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019']

In [65]:
# import north_china_plain boundary
Region =  ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone")\
            .filterMetadata('NAME1','equals',region_cn)

In [66]:
# Define the path to Sample_pt
sample_img_path = f'users/wangjinzhulala/China_built_up/04_sample_train_test_split'

In [67]:
export_path = 'users/wangjinzhulala/China_built_up/05_primary_classification'

##### Fetch all imgs

In [68]:
# read all asset_df into a list
asset_list = [pd.read_csv(f'../../File_trainsition/gee_asset_china/{str(i).zfill(2)}_path.csv') 
              for i in range(1,22)]

In [69]:
# concate all asset_df into one df
asset_df = pd.concat(asset_list)

In [70]:
asset_df.sort_values(['region','type','year'],inplace=True)

In [71]:
asset_df

,year,type,path,region
0,1990_1992,Fourier,projects/earthengine-legacy/assets/users/bthom...,dongbei
1,1993_1995,Fourier,projects/earthengine-legacy/assets/users/bthom...,dongbei
2,1996_1998,Fourier,projects/earthengine-legacy/assets/users/bthom...,dongbei
3,1999_2001,Fourier,projects/earthengine-legacy/assets/users/bthom...,dongbei
4,2002_2004,Fourier,projects/earthengine-legacy/assets/users/bthom...,dongbei
...,...,...,...,...
10,2005_2007,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/fhane...,zhongnan
11,2008_2010,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/fhane...,zhongnan
12,2011_2013,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/fhane...,zhongnan
13,2014_2016,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/fhane...,zhongnan


##### Prepare input_image for classification

Some additional treatment for the last two periods of Landsat and Sentinel imgs

In [72]:
# import Landsat img
Landsat_df = asset_df[(asset_df['type']=='Landsat_cloud_free')&(asset_df['region']==region_en)]
Landsat_img = [ee.Image(i) for i in  Landsat_df['path'].values]

In [73]:
# import Fourier img   
Fourier_df  = asset_df[(asset_df['type']=='Fourier')&(asset_df['region']==region_en)]
Fourier_img = [ee.Image(i) for i in  Fourier_df['path'].values]

In [74]:
# import Normalized img   
Normalized_df  = asset_df[(asset_df['type']=='NDVI_NDBI_EVI')&(asset_df['region']==region_en)]
Normalized_img = [ee.Image(i) for i in  Normalized_df['path'].values]

In [75]:
# prepare the climate data
Climate_mean = [ee.Image(f"users/China_bulit/01_Data_preparation/Climate_Mean_{year}")
                for year in year_name]

In [76]:
# Import DEM/SLOPE Img
DEM   = [ee.Image("USGS/SRTMGL1_003").rename('DEM')] * num_invarient
SLOPE = [ee.Terrain.slope(DEM).rename('SLOPE')] * num_invarient

In [77]:
# stack all Imput_Img together
Stack_img = [ee.Image(img) for img in zip(Landsat_img,
                                           Fourier_img,
                                           Normalized_img,
                                           Climate_mean,
                                           DEM,
                                           SLOPE)]

################ Some extra export for fig making, not useful for this classification ################

In [78]:
# # get the observation rectagles
# # clip = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/Observation_Rectangle_Full")

# # loop through each year to get target imgs
# for year,img in zip(year_name,Stack_img):
    
#     # get imgs
#     Fourier = img.select(['NDBI_cos_1','EVI_sin_2','NDVI_sin_2']).clip(clip)
#     Normalized = img.select(['Mean_NDVI','Mean_NDBI','Mean_EVI']).clip(clip)
    
#     # Landsat 8 has different flase color composite compare to 5/7
#     if int(year[-4:]) <= 2013 :
#         False_color = img.select(['B4','B3','B2']).clip(clip)
#     else:
#         False_color = img.select(['Landsat_B5','Landsat_B4','Landsat_B3']).clip(clip)
        
        
#     # stack imgs together for export    
#     imgs = [Fourier,Normalized,False_color] 
#     img_names = ['Fourier','Normalized','False_color'] 
    
#     # export to CloudStorage
#     for img,name in zip(imgs,img_names):
        
#         # construcnt img name for export
#         img_name = f'{name}_{year}'
        
#         # export
#         task = ee.batch.Export.image.toCloudStorage(image = img,
#                                                     description=img_name,
#                                                     bucket='north_china_plain',
#                                                     fileNamePrefix=img_name,
#                                                     region=clip,
#                                                     scale=30,
#                                                     maxPixels=int(1e13),
#                                                     skipEmptyTiles=True) 
    
#         task.start()
        
#         # print out the process.
#         print(f'Exporting {img_name} successful!')
    

##### Prepare input sample points

In [79]:
# Get the training sample
Training_sample = [ee.FeatureCollection(f"{sample_img_path}/Training_sample_{region_en}_{year}")
                   for year in year_name]

##### Stack input_img and input_sample together

In [80]:
# first put all inputs together
stack_year   = year_name
stack_img    = Stack_img 
stack_sample = Training_sample 

# stack all ingredients together
Stack_img_sample = list(zip(stack_year,stack_img,stack_sample))

#### Create the classification instances

In [81]:
# import the inbands-combination 
in_band_df = pd.read_csv('../Sub_Process_7_Before_classification_Feature_selection/Result/In_bands_combination.csv')

# selections for in-band combination
in_band_selection =[#'Spectrum',
                    #'Spectrum_Normalize',
                    #'Spectrum_Normalize_Fourier',
                    #'Spectrum_Normalize_Fourier_Terrain',
                    'Spectrum_Normalize_Fourier_Terrain_Meterology']

In [82]:
# A three-layer iteration to loopthrough [inbands/year/10-randome-split] individualy

Classificatioin_result = []

# The first-layer loop to go througth each inbands
for input_bands in in_band_selection:

    # The second-layer loop to go througth each year-periods
    for i,input_variable in enumerate(Stack_img_sample):
        
        
        # Fetch basic parameters
        classificaiton_tpye = input_bands
        year                = input_variable[0]
        input_img           = input_variable[1]
        sample_pt           = input_variable[2]

        # here determine which bands are involved in the classification
        # because the list is a string represented format, here use ast 
        # module to conver it back to a Python list
        in_features         = ast.literal_eval(in_band_df.at[i,input_bands])
        

        # The third-layer to go through each-randomnes
        for seed in range(10):

            #_______________________________Deploy the random-forest classification______________________
            # Instatiate the class with a name.
            classification = Classification(year_name      = year,
                                            Verified_point = sample_pt,
                                            Input_img      = input_img,
                                            Input_band     = in_features,
                                            Tree_num       = 100,
                                            seed           = seed,
                                            classProperty  = 'Built')


            # Get the classified img
            classified_img = classification.classification_img


            # Get the classified samples
            train_sample_classified = classification.Train_sample_classification
            test_sample_classified  = classification.Test_sample_classification

            #__________________________________Add all classification result into a list____________________________
            Classificatioin_result.append((classificaiton_tpye,year,seed,classified_img,test_sample_classified))

In [83]:
# convert classificaiton instances into a dataframe
Classification_instances_df =  pd.DataFrame(Classificatioin_result,
                                            columns=['classificaiton_tpye','year','seed','classified_img',
                                                     'test_sample_classified'])
Classification_instances_df

,classificaiton_tpye,year,seed,classified_img,test_sample_classified
0,Spectrum_Normalize_Fourier_Terrain_Meterology,1990_1992,0,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
1,Spectrum_Normalize_Fourier_Terrain_Meterology,1990_1992,1,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
2,Spectrum_Normalize_Fourier_Terrain_Meterology,1990_1992,2,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
3,Spectrum_Normalize_Fourier_Terrain_Meterology,1990_1992,3,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
4,Spectrum_Normalize_Fourier_Terrain_Meterology,1990_1992,4,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
...,...,...,...,...,...
95,Spectrum_Normalize_Fourier_Terrain_Meterology,2017_2019,5,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
96,Spectrum_Normalize_Fourier_Terrain_Meterology,2017_2019,6,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
97,Spectrum_Normalize_Fourier_Terrain_Meterology,2017_2019,7,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
98,Spectrum_Normalize_Fourier_Terrain_Meterology,2017_2019,8,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."


#### Add one of the classified img to map

In [84]:
# define parameters of classification for show in map
bands_in_type = 'Spectrum_Normalize_Fourier_Terrain_Meterology'
year_in       = '2017_2019'


# get the classification img
one = Classification_instances_df[(Classification_instances_df['classificaiton_tpye'] == bands_in_type) &
                                  (Classification_instances_df['year'] == year_in) &
                                  (Classification_instances_df['seed'] == 5)]['classified_img'].values[0]

In [85]:
# add the selected classification in map
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.centerObject(Region,6)
Map.addLayer(one,{'min':0,'max':1},'One')
Map

Map(center=[27.33041569106484, 111.63607653815916], controls=(WidgetControl(options=['position'], widget=HBox(…

##### Export the classified_img

In [86]:
for idx,item in Classification_instances_df.iterrows():
     
    Classification_type = item[0]
    year = item[1]
    seed = item[2]
    img  = item[3].toInt8()
    
    
    export_name = f'{Classification_type}_{region_en}_{year}_{seed}'
    
    #export to GEE asset    
    task = ee.batch.Export.image.toAsset(   image          = img,
                                            description    = export_name,
                                            assetId        = f'{export_path}/{export_name}',
                                            region         = Region.geometry().bounds(),
                                            scale          = 30,
                                            maxPixels      = int(1e13)) 
    
    task.start()
    
#     #export to Google Cloud Storage   
#     task = ee.batch.Export.image.toCloudStorage(    image = img,
#                                                     description=export_name,
#                                                     bucket='north_china_plain',
#                                                     fileNamePrefix=export_name,
#                                                     region=Region.geometry().bounds(),
#                                                     scale=30,
#                                                     maxPixels=int(1e13),
#                                                     skipEmptyTiles=True) 
    
#     task.start()
    
    print(f'Exporting {export_name} successful!')

Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_1990_1992_0 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_1990_1992_1 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_1990_1992_2 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_1990_1992_3 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_1990_1992_4 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_1990_1992_5 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_1990_1992_6 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_1990_1992_7 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_1990_1992_8 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_1990_1992_9 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_1993_1995_0 successful!
Exporting Spectrum_No

Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_2017_2019_3 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_2017_2019_4 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_2017_2019_5 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_2017_2019_6 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_2017_2019_7 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_2017_2019_8 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_zhongnan_2017_2019_9 successful!
